# <center> Social Distancing detector</center>

<center> <font size=4> Computer Vision & Internet of Things intern @ The Sparks Foundation </font></center> 

## <center> By: Md.Yeasin Sheikh </center>

### Necessary packages

In [63]:
from DistanceHelper.DistanceDetector import detectPeople
from DistanceHelper.DistanceDetector import MIN_CONF
from DistanceHelper.DistanceDetector import MIN_DISTANCE
from DistanceHelper.DistanceDetector import NMS_THRESH
from tqdm.notebook import tqdm
from scipy.spatial import distance as dist
import numpy as np
import imutils
import cv2
import time

In [64]:
videoFile = r'videos/testVideo_Trim.mp4'
configPath = r'YOLO_DB/yolov3.cfg'
weightsPath = r'YOLO_DB/yolov3.weights'
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

## Operation Selector

In [65]:
previewWhileProcess = False
saveLocally = True
writer = None

In [66]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print("we are going to use", ln)

['yolo_82', 'yolo_94', 'yolo_106']


<font size=4> init test envoirment</font>

In [67]:
cam = cv2.VideoCapture(videoFile)
fps = int(cam.get(cv2.CAP_PROP_FPS))
totalFPS = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))
computedFps = 0
print(fps, totalFPS)

30 149


List of detectable objects 

In [72]:
LABELS = []
with open("YOLO_DB/coco.names", 'rt') as f:
    LABELS = f.read().rstrip("\n").split('\n')
print("Number of objects Model hold: ", len(LABELS))
print("We gonna use person Only index of: ",LABELS.index("person"))

Number of objects Model hold:  80
We gonna use person Only index of:  0


<font size=4> Colorization </font>

In [69]:
color_violation = (0,0, 255)
color_safe = (0,255,0)

## Operation Center

In [70]:
print("[INFO] init Process..")
pbar = tqdm(total=totalFPS, desc="Progress") #progressBar
while True:
    success, frame = cam.read()
    
    if not success:
        print("[INFO] EOF Process..")
        break        
    
    frame = imutils.resize(frame, width=700)
    results = detectPeople(frame, net, ln, personIdx=LABELS.index('person'))


    voilation = set()

    if len(results) >=2:
        centroids = np.array([r[2] for r in results])
        distanceMap = dist.cdist(centroids, centroids, metric = "euclidean")

#   print(distanceMap.shape)
        for i in range(distanceMap.shape[0]):
            for j in range(i+1, distanceMap.shape[1]):

                #if distance between 2person is less minDistance
                if distanceMap[i, j] < MIN_DISTANCE:
                    voilation.add(i)
                    voilation.add(j)


        #Loop over results and make visualizations
    for (i, (prob, bbox, centroid)) in enumerate(results):
        (startX, startY, endX, endY)  = bbox
        (cX, cY) = centroid

        color = color_safe

        if i in voilation:
            color = color_violation

        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2) #bounding box
        cv2.circle(frame, (cX, cY), 5, color, 1)
    text = "Social Distancing Violations: {}".format(len(voilation))
    cv2.putText(frame, text, (10, frame.shape[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

    #should be display
    if previewWhileProcess:
        cv2.imshow("Social Distance", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    
    #init save file
    if saveLocally and writer is None:
#         print(size)
        fourcc =cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('videos/output.avi', fourcc, 20, (frame.shape[1], frame.shape[0]), True) 
    
    if writer is not None:
        writer.write(frame)
        
    #ProgressBar 
    computedFps +=1
    pbar.update(1)

pbar.close()
writer.release()     
cv2.destroyAllWindows()


[INFO] init Process..


Progress:   0%|          | 0/149 [00:00<?, ?it/s]

(1920, 1080)
[INFO] EOF Process..
